In [1]:
import pickle
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
def connect_db(username='root', password='root', host='localhost', port='5432', database='db_umls'):

    conn = None
    
    try:
        engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
        conn = engine.connect()
        
        if conn:
            print(f'Conectado correctamente a la base de datos: {database}')
            
    except Exception as e:
        print(f'Error al conectar a la base de datos: {e}')
        
    return conn, engine

In [3]:
conn, engine = connect_db()

Conectado correctamente a la base de datos: db_umls


In [4]:
def load_pkl(file_path):
    try:
        with open(file_path, 'rb') as file:
            df = pickle.load(file)
        print(f"Successfully loaded '{file_path}' from the file.")
        return df
    
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
        
    except Exception as e:
        print(f"Error: An exception occurred - {e}")
        return None

In [5]:
def poblar_db(engine):

    df = load_pkl(file_path='my_vocabulary.pkl')

    df.columns = df.columns.str.lower()
    
    df.to_sql('vocabulary_umls', engine, if_exists='replace', index=False)
    
    return print('Se cargaron {} registros en la base de datos'.format(len(df)))

In [6]:
def execute_poblar():
    conn, engine = connect_db()
    poblar_db(engine)
    
    if conn.close:
        print('Desconectado correctamente de la db')

In [7]:
execute_poblar()

Conectado correctamente a la base de datos: db_umls
Successfully loaded 'my_vocabulary.pkl' from the file.
Se cargaron 932493 registros en la base de datos
Desconectado correctamente de la db


In [8]:
def queries_db(query):

    conn, engine = connect_db()

    if conn:
        result = conn.execute(text(query))

        # Itera sobre los resultados e imprime cada fila
        for row in result:
            print(row)

        conn.close()  # Cierra la conexión cuando hayas terminado

In [9]:
query = """
        SELECT * FROM vocabulary_umls WHERE str = 'dolor de cabeza'
        """
queries_db(query)

query = """
        SELECT str, sty FROM vocabulary_umls WHERE str = 'dolor de cabeza'
        """
queries_db(query)

Conectado correctamente a la base de datos: db_umls
('C0018681', 'dolor de cabeza', 'T184', 'sign or symptom')
Conectado correctamente a la base de datos: db_umls
('dolor de cabeza', 'sign or symptom')
